Thinking1:在CTR点击率预估中，使用GBDT+LR的原理是什么？
答：GBDT+LR主要思想为集成学习，分为GBDT和LR两个部分，GBDT为一系列的分类树模型的组合，训练时，后面一棵树学习目标为前一棵树训练结果的残差，然后整个GBDT是通过多课弱分类器的数来提升训练效果，GBDT学习的结果是并不是CTR点击率，而是某条样本的激活特征以独热编码的方式表达的向量，然后这些向量再作为特征输入到LR模型中去训练LR模型，在应用中，需要将数据除了区分训练样本和测试样本外，还需要将训练样本一分为二，一本分用来训练GBDT，另一部分用来训练LR模型，以防两个训练样本交叉导致LR模型训练不充分，最终训练出的GBDT+LR模型才能用来做CTR点击率预估。

Thinking2:Wide & Deep的模型结构是怎样的，为什么能通过具备记忆和泛化能力（memorization and generalization）
答：Wide & Deep是结合线性模型的记忆能力和DNN深度模型的推理泛化能力的一种模型结构，线性模型能够能够记住历史出现的特征组合，根据用户已有的行为进行推荐，而深度模型能够挖掘未出现的隐藏特征，对历史没有出现的用户行为具有很好的泛化性，提高推荐物品的多样性。

Thinking3:在CTR预估中，使用FM与DNN结合的方式，有哪些结合的方式，代表模型有哪些？
答：包括FNN、DeepFM、NFM等，FNN是类似于DeepFM的部分，但是它的DNN部分的参数初始化是用的FM模型；DeepFM是从Wide & Deep优化来的，它将Wide & Deep中的Wide部分替换成了FM，它的FM部分和DNN部分是并行关系；而NFM中的FM的结果是DNN的输入，她的FM和DNN部分是串行关系。

Thinking4:GBDT和随机森林都是基于树的算法，它们有什么区别？
答：GBDT是以前一棵树的训练残差为训练目标，所有的树训练有先后顺序，是boosting的方式，是串行的训练方式，而随机森林采用的是bagging的方式，通过“少数服从多数”的方式来确定结果，树和树之间并没有先后顺序，不会相互影响，代码逻辑实现是并行的。

Thinking5:item流行度在推荐系统中有怎样的应用？
答：在冷启动中没有用户行为的情况下可以为用户推荐高流行度的item,在个性化推荐中，流行度高的item反而体现用户个性较少，在计算用户相似度时，如果共同item的流行度比较高则进行降权处理，这样去计算用户之间的相似度从而进行个性化推荐，挖掘用户对非高流行商品的需求。

In [ ]:
import pandas as pd
data=pd.read_csv("G:\\python_lesson\\L4-code\\L4\\MovieLens\\movielens_sample.txt")

In [30]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
sparse_features=['movie_id','user_id','gender','age','occupation','zip']
target=['rating']
train,test=train_test_split(data,test_size=0.2)

In [31]:
from sklearn.preprocessing import LabelEncoder
from deepctr.models import WDL
from deepctr.feature_column import SparseFeat,get_feature_names
for feature in features:
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature])
    #print(data[feature].nunique())
    #print(data[feature])
# 计算每个特征中的 不同特征值的个数
fixlen_feature_columns = [SparseFeat(feature, data[feature].nunique()) for feature in sparse_features]
#print(fixlen_feature_columns)
#print(type(fixlen_feature_columns))
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)
#print(feature_names)

In [32]:
train_model_input={name:train[name].values for name in feature_names}
test_model_input={name:test[name].values for name in feature_names}

In [33]:
model = WDL(linear_feature_columns, dnn_feature_columns, task='regression')
model.compile("adam", "mse", metrics=['mse'], )
history = model.fit(train_model_input, train[target].values, batch_size=256, epochs=1, verbose=True, validation_split=0.2, )
print(model.weights)
#print(history)
# # 使用WDL进行预测
pred_ans = model.predict(test_model_input, batch_size=256)
#print(pred_ans)
# # 输出RMSE或MSE
mse = round(mean_squared_error(test[target].values, pred_ans), 4)
rmse = mse ** 0.5
print("test RMSE", rmse)

1/1 [==============================] - 0s 444ms/step - loss: 14.2811 - mse: 14.2811 - val_loss: 13.3016 - val_mse: 13.3016
[<tf.Variable 'sparse_emb_movie_id/embeddings:0' shape=(187, 4) dtype=float32, numpy=
array([[ 9.52753238e-04,  8.72352801e-04,  1.05104258e-03,
        -9.93271009e-04],
       [-8.57541861e-04, -8.77777755e-04,  1.01470063e-03,
         1.00120774e-03],
       [ 6.73545364e-05, -1.51432696e-05, -1.61080490e-04,
        -8.35782485e-05],
       [-9.90714529e-04, -1.14680931e-03,  9.20404913e-04,
         1.03306992e-03],
       [ 1.05953822e-03, -1.15970813e-03, -1.06636283e-03,
         1.03486725e-03],
       [ 9.51307302e-04,  7.94055697e-04,  9.25506582e-04,
         1.11164059e-03],
       [ 9.65974410e-04,  1.05415832e-03,  8.58461019e-04,
         1.03012146e-03],
       [ 1.10794890e-05, -2.64780465e-05, -9.42672341e-05,
        -1.61406700e-04],
       [-4.60478259e-05, -2.44269730e-04, -6.33310847e-05,
        -5.15495631e-05],
       [ 1.13377813e-03,  